In [1]:
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [157]:
train_df = pd.read_excel("data/train.xlsx")
test1_df = pd.read_excel("data/testA.xlsx")
test2_df = pd.read_excel("data/testB.xlsx")

In [158]:
train_y = train_df[train_df.columns[-1]]
train_y.shape

(800,)

In [159]:
train_df = train_df[train_y>1.8]
train_y = train_y[train_y>1.8]
print(train_df.shape, train_y.shape)

(799, 5954) (799,)


In [160]:
test1y = pd.read_csv('data/testAy.csv', header=None)

In [161]:
test1y = test1y.values[:, 1]
test1y.shape

(300,)

In [162]:
new_column = test1_df.columns
train_df = train_df[new_column]
train_id = train_df['ID']
test1_id = test1_df['ID']
test2_id = test2_df['ID']
train_df.drop(['ID'], axis=1, inplace=True)
test1_df.drop(['ID'], axis=1, inplace=True)
test2_df.drop(['ID'], axis=1, inplace=True)

In [164]:
names = train_df.columns
dts = train_df.dtypes
drop_names = []
year = []
ratio = 0.15
for i in range(len(names)):
    n = names[i]
    tp = dts[i]
    trd = train_df[n]
    ted = test1_df[n]
    if 'object' == str(tp):
        continue
    else:
        if str(train_df[n][0])[:4] == '2017' and str(train_df[n][499])[:4] == '2017':
                year.append(n)
        else:
            misstr = np.sum(trd.isnull())/799
            misste = np.sum(ted.isnull())/300
            if np.abs(misstr-misste) > ratio:
                drop_names.append(n)
            
print(len(drop_names))
print(len(year))

1264
45


In [123]:
train_df.drop(drop_names, axis=1, inplace=True)
test1_df.drop(drop_names, axis=1, inplace=True)
test2_df.drop(drop_names, axis=1, inplace=True)

In [124]:
names = train_df.columns
dfs = train_df.dtypes
# ints or strings
ints = []
for i in range(len(names)):
    if 'int' in str(dfs[i]) or 'object' in str(dfs[i]):
        ints.append(names[i])
print(len(ints))

680


In [125]:
df_X = pd.concat([train_df, test1_df, test2_df], axis=0)

In [126]:
dfx_year = df_X[year]

In [127]:
remains1 = []
remains2 = []
tobedrop = []

for n in ints:
    lu1 = len(train_df[n].unique())
    lu2 = len(df_X[n].unique())
    if lu1 < 10 and lu1 > 1:
        remains1.append(n)
    if lu2 < 10 and lu2 > 1:
        remains2.append(n)
    if lu1 != lu2:
        tobedrop.append(n)
    
print(len(remains1), len(remains2), len(tobedrop), len(year))

159 222 317 40


In [128]:
toberemain = list(set(remains1) & set(remains2) - set(tobedrop))
print(len(toberemain))

84


In [129]:
def dict2list(dic:dict):
    ''' 将字典转化为列表 '''
    keys = dic.keys()
    vals = dic.values()
    lst = [(key, val) for key, val in zip(keys, vals)]
    return lst

def LastRemainJudge(column):
    unique = set(column)
    uf = {}
    for u in unique:
        uf[u] = 0
    for c in column:
        uf[c] += 1
    sl = sorted(dict2list(uf), key = lambda x:x[1], reverse=True)
    x = sl[0][1]/len(column)
    sl = np.array(sl)
    if x < 0.8:
        #print(sl[:, 1]/len(column))
        return True
    else:
        return False

In [130]:
lm1 = []
lm2 = []
for n in toberemain:
    if LastRemainJudge(train_df[n]):
        lm1.append(n)
    if LastRemainJudge(df_X[n]):
        lm2.append(n)
print(len(lm1), len(lm2), len(set(lm1)&set(lm2)))

53 52 52


In [131]:
names = df_X.columns
dts = df_X.dtypes
dfx_float_n = []
dfx_onehot_n = list(set(lm1) & set(lm2))
print(len(dfx_onehot_n))
for i in range(len(names)):
    n = names[i]
    d = dts[i]
    if 'float' in str(d):
        dfx_float_n.append(n)
print(len(dfx_float_n))
dfx_float = df_X[dfx_float_n]
dfx_onehot = df_X[dfx_onehot_n]
print(dfx_float.shape, dfx_onehot.shape)

52
3851
(1511, 3851) (1511, 52)


In [132]:
drop_names = []
names = dfx_float.columns
for n in names:
    x = np.sum(dfx_float[n].isnull())
    if x > 500:
        drop_names.append(n)
print(len(drop_names))

75


In [133]:
means = []
stds = []
drop_names = []
names = dfx_float.columns
for n in names:
    c = dfx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    means.append(tmpm)
    stds.append(tmps)
    if tmps == 0:
        drop_names.append(n)
print(len(set(means)), len(set(stds)))
print(len(drop_names))

1866 1761
346


In [134]:
dfx_float.drop(drop_names, axis=1, inplace=True)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [135]:
# not move sames first, fill nan
names = dfx_float.columns
added = set()
sames = []
for i in range(len(names)):
    tmpsl = []
    tmpsl.append(i)
    for j in range(i+1, len(names)):
        if means[i] == means[j] and stds[i] == stds[j]:
            if i not in added:
                added.add(i)
            if j not in added:
                added.add(j)
                tmpsl.append(j)
    if len(tmpsl) > 1:
        sames.append(tmpsl)
print(len(sames))

372


In [136]:
names = dfx_float.columns
drop_names = []
for i in sames:
    for j in i[1:]:
        drop_names.append(names[j])
print(len(drop_names))

1671


In [137]:
dfx_float.drop(drop_names, axis=1, inplace=True)
print(dfx_float.shape)

(1511, 1834)


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [138]:
cpdfxf = copy.copy(dfx_float)

In [139]:
change_names1 = []
change_names2 = []
change_names3 = []
for n in dfx_float.columns:
    c = dfx_float[n]
    r = c[c.notnull()]
    if len(set(r))<=3:
        continue
    x = np.sort(list(set(r)))
    max1 = x[-1] - x[-2]
    max2 = x[-2] - x[-3]
    min1 = x[1] - x[0]
    min2 = x[2] - x[1]
    if (max1 > 0 and max2==0) or (min1>0 and min2==0):
        change_names1.append(n)
    else:
        if (max2>0 and max1/max2>=7.3891):
            change_names2.append(n)
            newmax1 = x[-2] + max2
            dfx_float[n][c.notnull()][r==x[-1]] = newmax1
        if (min2>0 and min1/min2>=7.3891):
            change_names3.append(n)
            newmin1 = x[1] - min2
            dfx_float[n][c.notnull()][r==x[0]] = newmin1
print(len(change_names1), len(change_names2), len(change_names3))

0 217 338


In [140]:
names = dfx_float.columns
for n in names:
    c = dfx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    assert tmps>0
    dfx_float[n][c.isnull()] = tmpm
print(np.sum(dfx_float.isnull().values))

/home/magnusterra/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/magnusterra/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


0


In [141]:
# it shall be that there is only thre lines with nan in log1p
mid = dfx_float-np.min(dfx_float)+1
dfx_log1p = np.log1p(mid)
del mid
print(np.sum(dfx_log1p.isnull().values))

0


In [142]:
for n in names:
    m1 = np.mean(dfx_float[n])
    s1 = np.std(dfx_float[n])
    assert s1 > 0
    dfx_float[n] = (dfx_float[n]-m1)/s1
    m2 = np.mean(dfx_log1p[n])
    s2 = np.std(dfx_log1p[n])
    assert s2 > 0
    dfx_log1p[n] = (dfx_log1p[n]-m2)/s2

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [143]:
# make onehots onehot
ohnames = dfx_onehot.columns
for i in range(len(ohnames)):
    n = ohnames[i]
    tmp = pd.get_dummies(dfx_onehot[n], prefix='onehot_'+str(i))
    dfx_onehot = pd.concat([dfx_onehot, tmp], axis=1)
print(dfx_onehot.shape)

(1511, 252)


In [144]:
dfx_onehot.shape

(1511, 252)

In [145]:
dfx_onehot.drop(ohnames, axis=1, inplace=True)
print(dfx_onehot.shape)

(1511, 200)


In [146]:
year_n = dfx_year.columns
standardlen = len(str(dfx_year[year_n[0]][0]))
for i in range(1, len(year_n)):
    l = len(str(dfx_year[year_n[i]][0]))
    bias = standardlen - l
    b10 = 1
    if bias > 0:
        for j in range(bias):
            b10 *= 10
            #dfx_year[year_n[i]] /= 10
    elif bias < 0:
        for j in range(-bias):
            b10 /= 10
            #dfx_year[year_n[i]] *= 10
    if b10 != 1:
        dfx_year[year_n[i]] *= b10
    dfx_year[year_n[i]] = dfx_year[year_n[i]] - dfx_year[year_n[0]] 

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [147]:
dfx_time = (dfx_year - np.mean(dfx_year))/np.std(dfx_year)
dfx_time.shape
dfx_time = dfx_time[dfx_time.columns[1:]]
dfx_time.shape

(1511, 39)

In [148]:
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance

In [149]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

In [150]:
def MSE(y_raw, y_pred):
    r = np.mean(np.square(y_raw-y_pred))
    #print(r)
    return r

In [151]:
dfx = pd.concat([dfx_log1p, dfx_onehot], axis=1)
dfx.shape

(1511, 2034)

In [152]:
ndfx = pd.concat([dfx, dfx_time], axis=1)
ndfx.shape

(1511, 2073)

In [155]:
xgb = XGBRegressor(gamma=0.0001, max_depth=1, objective='reg:linear')
xgb.fit(ndfx[:799].values, train_y.values)
test1_pred = xgb.predict(ndfx[:799].values)
print(MSE(test1_pred, train_y.values))
test1_pred = xgb.predict(ndfx[799:1099].values)
print(MSE(test1_pred, test1y))

0.0154202196647
0.03883532748316096
